In [2]:
# Imports
import numpy as np
import itertools
from IPython.display import display, Markdown, Latex, Math

%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from matplotlib.patches import Circle, Wedge
from matplotlib.collections import PatchCollection

import tropical
import data_gen2

import itertools

import time

from scipy.spatial import ConvexHull, Delaunay
from scipy.spatial.qhull import QhullError

from scipy.optimize import linprog

In [3]:
def evaluate_trop(polynomial, data):

    temp = np.ones((data.shape[0], 1))
    test = np.concatenate((data, temp), axis=1)
    
    Fresult = np.matmul(polynomial, test.T)
    Fmax = np.amax(Fresult, axis=0)
    Flocs = np.argmax(Fresult, axis=0)
    
    return Fmax, Flocs

In [7]:
# Hyperparameters

# Starting F and G polynomials

nTerms = 2

F1 = np.random.uniform(low=-1, high=1, size=(nTerms, 3))
G1 = np.random.uniform(low=-1, high=1, size=(nTerms, 3))


structure = 'xor'

if structure == 'circles':
    num_samples = 4
    
    r1 = 0.5 # Radius of inner circle
    data_func = lambda n: data_gen2.circles(n, r1)
elif structure == 'xor':
    num_samples = 2
    
    r1 = 0.25
    data_func = lambda n: data_gen2.harderXor(n, r1=r1, alpha=np.sqrt(2)-1)

In [27]:
# vars = [epsilon, z_i, w_1, b_1, w_2, b_2, ...]

data, _, _ = data_func(num_samples)
x = data[:, 0:2]
y = data[:, -1]

nVars = num_samples*2 + nTerms*3
nRows = num_samples*nTerms

c = np.zeros((nVars))
c[:num_samples] = 1
c[num_samples:2*num_samples] = -1

A_ub = np.zeros((nRows, nVars))
b_ub = np.zeros((nRows))
Gresult, _ = evaluate_trop(G1, x)

for i in range(num_samples):
    temp = np.zeros((nTerms, nVars))
    temp[:, i] = 1
    temp[:, num_samples + i] = -1
    for j in range(nTerms):
        temp[j, 2*num_samples + j*3] = x[i, 0]
        temp[j, 2*num_samples + j*3+1] = x[i, 1]
        temp[j, 2*num_samples + j*3+2] = 1
    
    A_ub[nTerms*i:nTerms*(i+1), :] = temp
    b_ub[nTerms*i:nTerms*(i+1)] = y[i] + Gresult[i]


bounds = [(0, None) for i in range(nVars)]
print(A_ub)
print(b_ub)
print(c)
print(bounds)
res = linprog(c=c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)
print(res['message'])

[[ 1.          0.         -1.          0.          0.15266849  0.69987286
   1.          0.          0.          0.        ]
 [ 1.          0.         -1.          0.          0.          0.          0.
   0.15266849  0.69987286  1.        ]
 [ 0.          1.          0.         -1.         -0.56817213 -0.43039088
   1.          0.          0.          0.        ]
 [ 0.          1.          0.         -1.          0.          0.          0.
  -0.56817213 -0.43039088  1.        ]]
[-1.06323732 -1.06323732  1.8543687   1.8543687 ]
[ 1.  1. -1. -1.  0.  0.  0.  0.  0.  0.]
[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]
Optimization failed. The problem appears to be unbounded.
